In [5]:
!pip install sweetviz
!pip install plotly
!pip install ydata_profiling
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [6]:
# Importar librerías
## EDA
import pandas as pd
import numpy as np
import sweetviz as sv
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## Modelamiento
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from catboost import CatBoostClassifier
from scipy.stats import randint, uniform

# a. Análisis Exploratorio

In [7]:
# Cargar datos
clientes = pd.read_excel('clientes_final.xlsx')
ventas = pd.read_excel('venta_final.xlsx')

## Variables

In [ ]:
# Exploración inicial
print("\nClientes DataFrame Info:")
print(clientes.info())

print("\nVentas DataFrame Info:")
print(ventas.info())

# Estadísticas descriptivas
print("\nEstadísticas descriptivas - Clientes:")
print(clientes.describe())

print("\nEstadísticas descriptivas - Ventas:")

In [ ]:
print(ventas.drop_duplicates().shape)
print(clientes.drop_duplicates().shape)

Se asume que los registros deben ser únicos en cada base, por lo tanto, que tanto en la base de clientes como en la de ventas solo existe un registro por cliente y por venta (con la misma información)

In [ ]:
ventas.drop_duplicates(inplace=True)
clientes.drop_duplicates(inplace=True)

## Clientes

In [ ]:
ventas['porc_desc'] = abs(ventas['desc'])/(abs(ventas['desc'])+ ventas['nr'])
ventas['porc_desc_sin_0'] = np.where(ventas['porc_desc'] == 0, np.nan, ventas['porc_desc'])

In [ ]:
ventas['vol_cat'] = np.where((ventas['vol'] > 1)&(ventas['vol'] <= 1.5), '(1-1.5]',
                           np.where((ventas['vol'] > 1.5)&(ventas['vol'] <= 2.5), '(1.5-2.5]',
                                    np.where((ventas['vol'] > 2.5)&(ventas['vol'] <= 9.5), '(2.5-9.5]', '>9.5')))
ventas['nr_cat'] = np.where((ventas['nr'] == 0), '0',
                           np.where((ventas['nr'] > 0)&(ventas['nr'] <= 2600000), '(0-2.6M]',
                                    np.where((ventas['nr'] > 2600000)&(ventas['nr'] <= 3500000), '(2.6M-3.5M]',
                                             np.where((ventas['nr'] > 3500000)&(ventas['nr'] <= 34000000), '(3.5M-34M]', '>34M'))))
ventas['porc_desc_cat'] = np.where((ventas['porc_desc'] == 0), '0%',
                                 np.where((ventas['porc_desc'] > 0)&(ventas['porc_desc'] <= 0.2), '(0%-20%]',
                                          np.where((ventas['porc_desc'] > 0.2)&(ventas['porc_desc'] <= 0.4), '(20%-40%]',
                                                   np.where((ventas['porc_desc'] > 0.4)&(ventas['porc_desc'] <= 0.6), '(40%-60%]',
                                                            np.where((ventas['porc_desc'] > 0.6)&(ventas['porc_desc'] <= 0.8), '(60%-80%]',
                                                                     np.where((ventas['porc_desc'] > 0.8)&(ventas['porc_desc'] <= 1), '(80%-100%]', 'No_Aplica'))))))

In [8]:
abt = pd.merge(ventas, clientes, on='Cliente', how='left')

In [ ]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)
abt.groupby(['Gerencia', 'subcanal']).agg({'Cliente': 'nunique',
                                           'porc_desc': ['min', 'max', 'mean'],
                                           'nr': ['min', 'max', 'mean', 'sum'],
                                           'vol': ['min', 'max', 'mean', 'sum']})

## Insights

In [ ]:
report =  sv.analyze(abt)
report.show_html('EDA_Fuentes.html')

In [ ]:
profile = ProfileReport(abt, title="EDA Fuentes Clientes & Ventas", minimal=True)
profile.to_file("EDA_Fuentes_2.html")

In [ ]:
# Gráfico de dispersión productos vendidos vs. ganancia
fig = px.scatter(abt, x='vol', y='nr', color='porc_desc_cat',
                 color_continuous_scale='viridis',
                 title='Gráfica de Cantidad de productos vendidos vs. Ganancia, por grupo de descuento',
                 labels={'vol': 'Cantidad de productos vendidos',
                         'nr': 'Ganancia',
                         'porc_desc_cat': 'Porcentaje de descuento'})

fig.show()

In [ ]:
abt.groupby(['Cliente', 'ym']).agg({'brand': 'count'}) #Hay más de un registro por cliente y mes

In [ ]:
suma_nr = abt.groupby(['brand'])['nr'].sum()
suma_vol = abt.groupby(['brand'])['vol'].sum()
ganan_x_und = pd.DataFrame({
    'suma_nr': suma_nr,
    'suma_vol': suma_vol
 })
ganan_x_und['ganancia_x_und'] = ganan_x_und['suma_nr'] / ganan_x_und['suma_vol']

In [ ]:
# Proporción de ventas por marca
pd.concat([(100*abt.groupby(['brand']).agg({'Cliente': 'count'})/len(abt)).sort_values(by='Cliente'),
           (abt.groupby(['brand']).agg({'Cliente': 'count'})),
           ganan_x_und['ganancia_x_und']], axis=1)

In [ ]:
# Proporción de ventas por segmento
pd.concat([(100*abt.groupby(['segment']).agg({'Cliente': 'count'})/len(abt)).sort_values(by='Cliente'),
           (abt.groupby(['segment']).agg({'Cliente': 'count'})).sort_values(by='Cliente')], axis=1)

# b. Modelo de venta - Marca1

In [9]:
# Marca elegida: Marca1
abt_final = abt[['Gerencia', 'subcanal', 'desc', 'brand', 'ym', 'Cliente', 'vol']]
abt_final['target'] = np.where(abt['brand'] == 'Marca1', 1, 0)
abt_final

C:\Users\ivfen\AppData\Local\Temp\ipykernel_3976\3468331107.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abt_final['target'] = np.where(abt['brand'] == 'Marca1', 1, 0)


,Gerencia,subcanal,desc,brand,ym,Cliente,vol,target
0,Gerencia1,tiendita,0.000000e+00,Marca1,202301,cte1,4.877,1
1,Gerencia1,tiendita,0.000000e+00,Marca1,202302,cte1,2.519,1
2,Gerencia1,tiendita,-1.863603e+06,Marca2,202302,cte1,2.092,0
3,Gerencia1,tiendita,0.000000e+00,Marca1,202303,cte1,3.633,1
4,Gerencia1,tiendita,0.000000e+00,Marca2,202303,cte1,1.696,0
...,...,...,...,...,...,...,...,...
186898,Gerencia5,tiendita,0.000000e+00,Marca7,202406,cte2859,1.854,0
186899,Gerencia5,tiendita,0.000000e+00,Marca2,202406,cte2859,1.854,0
186900,Gerencia2,tiendita,0.000000e+00,Marca2,202406,cte2860,1.430,0
186901,Gerencia1,tiendita,0.000000e+00,Marca1,202406,cte2861,1.399,1


In [10]:
# Creación de variable de volumen vendido anteriormente al cliente de esta misma marca
abt_final['ym'] = pd.to_datetime(abt_final['ym'], format='%Y%m')
df_marca1 = abt_final[abt_final['brand'] == 'Marca1'].copy()
df_marca1 = df_marca1.sort_values(by=['Cliente', 'ym'])
df_marca1['vol_vendido_antes_m1'] = df_marca1.groupby('Cliente')['vol'].cumsum() - df_marca1['vol']
result = abt_final.merge(df_marca1[['Cliente', 'ym', 'vol_vendido_antes_m1']],
                         on=['Cliente', 'ym'],
                         how='left')
result['vol_vendido_antes_m1'] = result['vol_vendido_antes_m1'].fillna(0)

# Creación del descuento dado al cliente en los periodos anteriores
df_desc = result.loc[result['brand'] == 'Marca1']
df_desc['abs_desc'] = abs(df_desc['desc'])
df_desc = df_desc.sort_values(by=['Cliente', 'ym'])
df_desc['desc_antes_m1'] = df_desc.groupby('Cliente')['abs_desc'].cumsum() - df_desc['abs_desc']
result2 = result.merge(df_desc[['Cliente', 'ym', 'desc_antes_m1']],
                         on=['Cliente', 'ym'],
                         how='left')
result2['desc_antes_m1'] = result2['desc_antes_m1'].fillna(0)

C:\Users\ivfen\AppData\Local\Temp\ipykernel_3976\982487056.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abt_final['ym'] = pd.to_datetime(abt_final['ym'], format='%Y%m')
C:\Users\ivfen\AppData\Local\Temp\ipykernel_3976\982487056.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_desc['abs_desc'] = abs(df_desc['desc'])


Se agrupan las ventas por cliente y fecha, para modelar si un cliente comprará en un mes la marca 1

In [11]:
abt_modelo = result2.groupby(['Cliente', 'ym', 'Gerencia', 'subcanal']).agg({'desc_antes_m1': 'max',
                                        'vol_vendido_antes_m1' : 'max',
                                        'target' : 'max'}).reset_index(drop=False).drop(columns=['Cliente',
                                                                                                 'ym'])
abt_modelo.describe(include='all')

,Gerencia,subcanal,desc_antes_m1,vol_vendido_antes_m1,target
count,33782,33782,3.378200e+04,33782.000000,33782.000000
unique,9,2,NaN,NaN,NaN
top,Gerencia2,tiendita,NaN,NaN,NaN
freq,8531,33774,NaN,NaN,NaN
mean,NaN,NaN,5.123652e+07,50.463272,0.825410
std,NaN,NaN,8.066415e+08,175.898916,0.379622
min,NaN,NaN,0.000000e+00,0.000000,0.000000
25%,NaN,NaN,0.000000e+00,1.404000,1.000000
50%,NaN,NaN,1.872428e+05,14.300000,1.000000
75%,NaN,NaN,6.261473e+06,42.349750,1.000000


In [12]:
X = abt_modelo[['desc_antes_m1', 'vol_vendido_antes_m1', 'Gerencia', 'subcanal']]
y = abt_modelo['target']
numeric_features = ['desc_antes_m1', 'vol_vendido_antes_m1']
categorical_features = ['Gerencia', 'subcanal']

numeric_transformer = MinMaxScaler()
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

X_processed =preprocessor.fit_transform(X)

### Regresión Logística

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=0)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Obtener probabilidades de predicción
y_probs = model.predict_proba(X_test)[:, 1]

# Definir una serie de umbrales
thresholds = np.arange(0.0, 1.1, 0.1)

# Inicializar listas para almacenar métricas
precisions = []
recalls = []
f1_scores = []

# Evaluar el modelo para cada umbral
for threshold in thresholds:
    y_pred = (y_probs >= threshold).astype(int)

    # Calcular métricas
    conf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1_score)

# Graficar resultados
plt.figure(figsize=(5, 3))
plt.plot(thresholds, precisions, label='Precision', marker='o')
plt.plot(thresholds, recalls, label='Recall', marker='o')
plt.plot(thresholds, f1_scores, label='F1 Score', marker='o')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Threshold Analysis')
plt.legend()
plt.grid(True)
plt.show()

# Graficar la curva ROC
fpr, tpr, _ = roc_curve(y_test, y_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(5, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

In [ ]:
for threshold in [0.7, 0.8, 0.9]:
  y_probs = model.predict_proba(X_test)[:, 1]
  y_pred = (y_probs >= threshold).astype(int)
  conf_matrix = confusion_matrix(y_test, y_pred)
  print(conf_matrix)
  print(classification_report(y_test, y_pred))

### Importancia de variables

In [ ]:
feature_names = (numeric_features +
                 list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)))
coef = model.coef_[0]
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coef
}).sort_values(by='Coefficient', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

## Catboost

In [13]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ],
    remainder='passthrough'
)
X_processed = preprocessor.fit_transform(X[numeric_features])
X_categorical = X[categorical_features]
X_final = np.hstack((X_processed, X_categorical))

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=0)

# Ajustar el modelo CatBoost
cat_features = [2,3]
iterations = None 
early_stopping_rounds=None
learning_rate = None
depth = None

# Crear y entrenar el modelo
model = CatBoostClassifier(cat_features = cat_features,
                          iterations = iterations, 
                          early_stopping_rounds = early_stopping_rounds,
                          learning_rate = learning_rate,
                          depth = depth)
model.fit(X_train, y_train, cat_features = cat_features,
          verbose=200)

# Predicciones y evaluación
y_pred = model.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Learning rate set to 0.042102
0:	learn: 0.5725793	total: 86ms	remaining: 1m 25s
200:	learn: 0.1304761	total: 12.7s	remaining: 50.4s
400:	learn: 0.1301452	total: 25.8s	remaining: 38.5s
600:	learn: 0.1300018	total: 39.6s	remaining: 26.3s
800:	learn: 0.1300017	total: 52.9s	remaining: 13.2s
999:	learn: 0.1300016	total: 1m 5s	remaining: 0us
Confusion Matrix:
[[1128   65]
 [ 354 5210]]

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.95      0.84      1193
           1       0.99      0.94      0.96      5564

    accuracy                           0.94      6757
   macro avg       0.87      0.94      0.90      6757
weighted avg       0.95      0.94      0.94      6757



### Importancia de variables

In [22]:
import pickle
with open('model.pkl', 'wb') as archivo:
    pickle.dump(model, archivo)

In [ ]:
feature_importances = model.get_feature_importance()
feature_names = (numeric_features +
                 list(X_categorical.columns))

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importance:")
print(importance_df)

plt.figure(figsize=(5, 3))
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Importancia de las Características')
plt.show()

## 🎯 Maqueta del Prototipo

La presente maqueta representa un prototipo analítico orientado a la visualización y comprensión de los datos de ventas de Bavaria. Este prototipo está diseñado para responder preguntas estratégicas como:

- ¿Qué productos generan mayor volumen y rentabilidad?
- ¿Qué segmentos presentan mayores descuentos y cómo se comportan?
- ¿Cómo se distribuyen las ventas por categoría y región?

### Componentes visuales clave:

- **Gráfico de dispersión**: Volumen vs. rentabilidad (`plotly.express`) para identificar productos estrella
- **Mapas de calor y gráficos de barras**: Tendencias por categorías, precios y promociones
- **Perfiles automáticos de datos**: Sweetviz y YData Profiling para análisis exploratorio automático

Esta maqueta servirá como base para desarrollar un dashboard más completo e interactivo, o un informe automatizado para el equipo de decisiones de negocio.
